In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "YOUR_REAL_KEY"

In [ ]:
import google.generativeai as genai
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
print("API configured successfully")

In [ ]:
!pip install -q google-genai

In [ ]:
for m in client.models.list():
    print(m.name)

In [ ]:
from google import genai
import os

client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="Explain machine learning in simple words"
)

print(response.text)

In [ ]:
#Install Libraries
!pip install requests beautifulsoup4 sentence-transformers faiss-cpu openai

In [ ]:
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [ ]:
# step1::Web Scraping
url = "https://en.wikipedia.org/wiki/Machine_learning"

headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Extract only main content area
content = soup.find("div", {"id": "mw-content-text"})

# Remove unwanted tags
for script in content(["script", "style", "sup", "table"]):
    script.decompose()

text = content.get_text(separator=" ")

print(text[:1000])

In [ ]:
# step2::text into chunks
def chunk_text(text, chunk_size=500):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i:i+chunk_size])
    return chunks

chunks = chunk_text(text)

print("Total chunks:", len(chunks))
print("First chunk preview:\n", chunks[0])

In [ ]:
# step3::Generate Embeddings

model = SentenceTransformer("all-MiniLM-L6-v2")

# Convert chunks into embeddings
embeddings = model.encode(chunks)

print("Embedding shape:", embeddings.shape)

In [ ]:
# step4::Create FAISS Vector Database
dimension = embeddings.shape[1]

# Create FAISS index
index = faiss.IndexFlatL2(dimension)

# Add embeddings to index
index.add(np.array(embeddings))

print("Total vectors stored in FAISS:", index.ntotal)

In [ ]:
# step 5::Ask Question & Retrieve
question = "What is machine learning?"

# Convert question to embedding
question_embedding = model.encode([question])

# Retrieve top 3 similar chunks
k = 3
distances, indices = index.search(np.array(question_embedding), k)

retrieved_chunks = [chunks[i] for i in indices[0]]

print("Retrieved Chunks:\n")

for i, chunk in enumerate(retrieved_chunks):
    print(f"Chunk {i+1}:\n")
    print(chunk[:500])
    print("\n--------------------\n")

In [ ]:
!pip install google-generativeai

In [ ]:
print(question)

In [ ]:
#step 6::Connect to OpenAI
import google.generativeai as genai
import os

# Configure API
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Load model
model_llm = genai.GenerativeModel("gemini-2.5-flash")

# Combine retrieved chunks
context = " ".join(retrieved_chunks)

prompt = f"""
Answer the question using ONLY the context below.
If the answer is not in the context, say you don't know.

Context:
{context}

Question:
{question}
"""

response = model_llm.generate_content(prompt)

print("Final Answer:\n")
print(response.text)

In [ ]:
print("Sources:")
for i, chunk in enumerate(retrieved_chunks):
    print(f"Source chunk {i+1}")